<a href="https://colab.research.google.com/github/nssn96/ML_GestureRecognition/blob/main/ML_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 4 - Gesture Recognition using HMM(Hidden Markov models)


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [8]:
# #install hmmlearn using pip3
# !pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 15.6 MB/s 


In [9]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import random
import pickle
from hmmlearn import hmm

In [ ]:
#References Used


In [11]:
#Importing the dataset
dataset = pd.read_csv('Original_Dataset.csv')
print(dataset['LC_X_1'])

0      307
1      340
2      260
3      338
4      319
      ... 
345      0
346    301
347    298
348    278
349    279
Name: LC_X_1, Length: 350, dtype: int64
